# Credit Decision Engine (Unified SHAP Architecture)
A production-grade decision layer with feature-level explainability across all model layers.

In [3]:
import pandas as pd
import numpy as np
import shap
import joblib
import json
import warnings
from collections import defaultdict

warnings.filterwarnings('ignore')

In [4]:
# Load all models from their respective artifact folders
pd_model = joblib.load('../2*. Models/2. PD_model/artifacts/pd_model.joblib')
pd_scaler = joblib.load('../2*. Models/2. PD_model/artifacts/pd_scaler.joblib')
pd_features = joblib.load('../2*. Models/2. PD_model/artifacts/pd_features.joblib')

iso_model = joblib.load('../2*. Models/1. Anomaly_model/artifacts/isolation_forest.joblib')
if_scaler = joblib.load('../2*. Models/1. Anomaly_model/artifacts/if_scaler.joblib')
if_features = ['avgMonthlyIncome', 'incomeCV', 'expenseRatio', 'emiRatio', 'avgMonthlyBalance', 'bounceCount']

risk_model = joblib.load('../2*. Models/3. Risk-Label_model/artifacts/risk_random_forest.joblib')
risk_features = joblib.load('../2*. Models/3. Risk-Label_model/artifacts/risk_features.joblib')

hybrid_model = joblib.load('../2*. Models/4*. Hybrid-CreditScore_model/artifacts/hybrid_credit_score_model.joblib')
hybrid_features = joblib.load('../2*. Models/4*. Hybrid-CreditScore_model/artifacts/hybrid_features.joblib')

q_table = joblib.load('../2*. Models/5. RL_model/artifacts/q_learning_model.joblib')
q_bins = joblib.load('../2*. Models/5. RL_model/artifacts/q_learning_bins.joblib')
q_features = joblib.load('../2*. Models/5. RL_model/artifacts/q_learning_features.joblib')


bg_data = pd.read_csv('../3. Data/1. Raw_Features/features_only.csv') # Background Data

X_pd_all = pd_scaler.transform(bg_data[pd_features])
bg_data['PD'] = pd_model.predict_proba(X_pd_all)[:, 1] #Store PD in bg_data

X_if_all = if_scaler.transform(bg_data[if_features])
if_scores = iso_model.decision_function(X_if_all) # Gets anomaly score
bg_data['anomalyFlag'] = (if_scores < -0.05).astype(int) # 1 = anomaly, 0 = normal

X_pd_bg = pd_scaler.transform(bg_data[pd_features])
X_if_bg = if_scaler.transform(bg_data[if_features])
X_risk_bg = bg_data[risk_features]
X_hybrid_bg = bg_data[hybrid_features]

bg_data.head()

,avgMonthlyIncome,incomeCV,expenseRatio,emiRatio,avgMonthlyBalance,bounceCount,accountAgeMonths,PD,anomalyFlag
0,320283.48,0.030,0.485,0.086,82434.68,0,72,0.008969,0
1,429134.95,0.045,0.335,0.189,356063.18,0,105,0.014418,0
2,292380.50,0.042,0.351,0.146,125629.67,0,118,0.009612,0
3,516978.70,0.041,0.344,0.247,544929.67,0,95,0.028849,0
4,403574.08,0.043,0.341,0.228,409525.91,0,79,0.009810,0


In [5]:
BUSINESS_MAPPING = {
    "avgMonthlyIncome": "Monthly Income Level",
    "incomeCV": "Income Volatility",
    "expenseRatio": "Monthly Expense Burden",
    "emiRatio": "Existing Debt Commitments (EMI)",
    "avgMonthlyBalance": "Liquidity Reserve",
    "bounceCount": "Historical Payment Bounces",
    "accountAgeMonths": "Banking Relationship Age",
    "PD": "Probability of Default Signal",
    "anomalyFlag": "Unusual Transaction Behavior",
    "anomaly": "Anomaly Signal Intensity",
    "HybridCreditScore": "Consolidated Credit Score"
}

### 🧠 SHAP Explainer Strategy

This document explains the SHAP explainer selection strategy for each model used in the credit risk architecture.

---

### 📌 Overview

| Model | Type | SHAP Explainer | Reason |
|-------|------|---------------|--------|
| PD Model | Logistic Regression | `LinearExplainer` | Exact + Fast for linear models |
| Isolation Forest | Tree-Based | `TreeExplainer` | Optimized for tree ensembles |
| Risk Random Forest | Tree Ensemble | `TreeExplainer` | Aggregates contributions across trees |
| Hybrid Credit Model | Mixed Pipeline | `shap.Explainer()` | Automatic backend detection |

---

In [6]:
pd_explainer = shap.LinearExplainer(pd_model, X_pd_bg)
if_explainer = shap.TreeExplainer(iso_model)
risk_explainer = shap.TreeExplainer(risk_model)
hybrid_explainer = shap.Explainer(hybrid_model, X_hybrid_bg)

### 🧠 Why RL Q-Table Requires `KernelExplainer`

- ❌ RL Policy (Q-table) is **NOT** a standard ML model  
- ❌ It has **no weights, no trees, no gradients**  
- ❌ It is **not parameterized**  
- ✅ It is simply a **lookup table (dictionary)**  

`KernelExplainer` :

- ✔ Works for any function
- ✔ No knowledge of internals needed

### Define Prototype of our RL Model And send to `KernelExplainer`


In [7]:
#Function is Behaive Like Model
#Function Take Parameter as X => Produce Result(Array)

def q_policy_func(X):
    results = []

    for row in X:
        #Create tuple
        s = (np.digitize(row[0], q_bins['pd']),
             np.digitize(row[1], q_bins['anom']),
             np.digitize(row[2], q_bins['cs']))
        #Find Q Values From Q Table    
        q_vals = q_table.get(s, np.zeros(4))
        #Result As a Max
        results.append(np.max(q_vals))
    
    return np.array(results)

# KernelExplainer only needs a prototype
#So we can send Demo 3 Rows only
rl_bg = np.array([[0.1, 0.1, 600], [0.5, 0.5, 400], [0.1, 0.8, 400]])
rl_explainer = shap.KernelExplainer(q_policy_func, rl_bg)

In [8]:
def top_shap_features(values, names, k=3):
    idx = np.argsort(np.abs(values))[::-1][:k]
    return [f"{BUSINESS_MAPPING.get(names[i], names[i])} ({values[i]:+.3f})" for i in idx]


> ### Top Factors 👉 Which customer details most affected this calculation?
> ### Drivers 👉 Which signals made the system label the customer as LOW, MEDIUM, or HIGH risk?
> ### Factors 👉 Which things changed the final credit score?
> ### Rationales 👉 Why the system finally said APPROVE or REJECT.

In [9]:
def get_decision_explanation(input_row, explain=True):
    df_input = pd.DataFrame([input_row])
    response = {}
    
    # 1. PD Layer
    X_pd = pd_scaler.transform(df_input[pd_features])
    pd_val = pd_model.predict_proba(X_pd)[0, 1]
    response["PD"] = {"value": round(float(pd_val), 4)}
    
    if explain:
        pd_exp = pd_explainer(X_pd)
        pd_shap_vals = pd_exp.values.flatten()
        response["PD"]["top_factors"] = top_shap_features(pd_shap_vals, pd_features)

    # 2. Anomaly Layer
    X_if = if_scaler.transform(df_input[if_features])
    if_score = iso_model.decision_function(X_if)[0]
    response["Anomaly"] = {"score": round(float(if_score), 4)}
    
    if explain:
        if_exp = if_explainer(X_if)
        if_vals = if_exp.values.flatten()
        response["Anomaly"]["top_factors"] = top_shap_features(if_vals, if_features)

    # 3. Risk Layer
    df_risk = df_input.copy()
    df_risk['PD'] = pd_val
    df_risk['anomalyFlag'] = 1 if if_score < -0.05 else 0
    X_risk = df_risk[risk_features]
    risk_idx = int(risk_model.predict(X_risk)[0])
    risk_label = ["LOW", "MEDIUM", "HIGH"][risk_idx]
    response["RiskLabel"] = {"label": risk_label}
    
    if explain:
        risk_exp = risk_explainer(X_risk)
        if risk_exp.values.ndim == 3: 
            risk_vals = risk_exp.values[0, :, risk_idx]
        else:
            risk_vals = risk_exp.values.flatten()
        response["RiskLabel"]["drivers"] = top_shap_features(risk_vals, risk_features)

    # 4. Hybrid Score Layer
    X_hyb = df_input[hybrid_features]
    score_val = hybrid_model.predict(X_hyb)[0]
    response["HybridScore"] = {"value": round(float(score_val), 1)}
    
    if explain:
        hyb_exp = hybrid_explainer(X_hyb)
        hyb_vals = hyb_exp.values.flatten()
        response["HybridScore"]["factors"] = top_shap_features(hyb_vals, hybrid_features)

    # 5. RL Action Layer (Recommended Policy)
    anom_norm = np.clip(1.0 - (if_score + 0.5), 0, 1)
    X_rl = np.array([[pd_val, anom_norm, score_val]])
    s = (np.digitize(X_rl[0][0], q_bins['pd']),
         np.digitize(X_rl[0][1], q_bins['anom']),
         np.digitize(X_rl[0][2], q_bins['cs']))
    
    action_idx = int(np.argmax(q_table.get(s, np.zeros(4))))
    actions = ["REJECT", "APPROVE_LOW", "APPROVE_MEDIUM", "APPROVE_HIGH"]
    response["RL_Recommendation"] = {"action": actions[action_idx]}
    
    if explain:
        rl_exp = rl_explainer.shap_values(X_rl, silent=True)
        rl_vals = rl_exp.flatten()
        response["RL_Recommendation"]["rationales"] = top_shap_features(rl_vals, q_features)

    return response


In [10]:
import matplotlib.pyplot as plt

def visualize_decision(input_row):
    """Generates comprehensive visual explanations for all 5 model layers."""
    df_input = pd.DataFrame([input_row])
    
    print("\n--- 1. Probability of Default (Waterfall) ---")
    X_pd = pd_scaler.transform(df_input[pd_features])
    pd_exp = pd_explainer(X_pd)
    pd_exp.feature_names = [BUSINESS_MAPPING.get(f, f) for f in pd_features]
    
    plt.figure(figsize=(10, 4))
    shap.plots.waterfall(pd_exp[0], show=False)
    plt.title("Probability of Default Logic")
    plt.show()
    
    print("\n--- 2. Anomaly Detection (Force Plot) ---")
    X_if = if_scaler.transform(df_input[if_features])
    if_exp = if_explainer(X_if)
    if_exp.feature_names = [BUSINESS_MAPPING.get(f, f) for f in if_features]
    
    shap.plots.force(if_exp.expected_value, if_exp.shap_values[0], feature_names=if_exp.feature_names, matplotlib=True, show=False)
    plt.show()

    print("\n--- 3. Risk Label (Class-Specific Waterfall) ---")
    # Re-compute derived inputs locally for visualization context
    pd_val = pd_model.predict_proba(X_pd)[0, 1]
    if_score = iso_model.decision_function(X_if)[0]
    
    df_risk = df_input.copy()
    df_risk['PD'] = pd_val
    df_risk['anomalyFlag'] = 1 if if_score < -0.05 else 0
    X_risk = df_risk[risk_features]
    
    risk_pred_idx = int(risk_model.predict(X_risk)[0])
    risk_label = ["LOW", "MEDIUM", "HIGH"][risk_pred_idx]
    
    risk_exp = risk_explainer(X_risk)
    risk_exp.feature_names = [BUSINESS_MAPPING.get(f, f) for f in risk_features]
    
    # Handle multi-class output extraction
    if risk_exp.values.ndim == 3:
        shap_vals_class = risk_exp[:, :, risk_pred_idx]
    else:
        shap_vals_class = risk_exp
        
    plt.figure(figsize=(10, 4))
    shap.plots.waterfall(shap_vals_class[0], show=False)
    plt.title(f"Risk Classification Logic (Class: {risk_label})")
    plt.show()

    print("\n--- 4. Hybrid Credit Score (Bar Plot) ---")
    X_hyb = df_input[hybrid_features]
    hyb_exp = hybrid_explainer(X_hyb)
    hyb_exp.feature_names = [BUSINESS_MAPPING.get(f, f) for f in hybrid_features]
    
    plt.figure(figsize=(10, 4))
    shap.plots.bar(hyb_exp[0], show=False)
    plt.title(f"Hybrid Credit Score Logic (Score: {hybrid_model.predict(X_hyb)[0]:.1f})")
    plt.show()

    print("\n--- 5. RL Recommendation (Summary Bar) ---")
    anom_norm = np.clip(1.0 - (if_score + 0.5), 0, 1)
    X_rl = np.array([[pd_val, anom_norm, hybrid_model.predict(X_hyb)[0]]])
    
    rl_shap_vals = rl_explainer.shap_values(X_rl, silent=True)
    rl_feature_names = ["Prob Default", "Anomaly Intensity", "Credit Score"]
    
    # Create a simple bar plot for RL logic
    plt.figure(figsize=(8, 3))
    plt.barh(rl_feature_names, rl_shap_vals[0])
    plt.title("RL Policy Drivers")
    plt.xlabel("SHAP Value Impact")
    plt.axvline(x=0, color='k', linestyle='--', linewidth=0.8)
    plt.tight_layout()
    plt.show()
    
print('✅ Visual Explainability (Graph) function ready for all 5 models.')

✅ Visual Explainability (Graph) function ready for all 5 models.


In [11]:
# Phase 9: Validate SHAP Sanity
elite_customer = {
    'avgMonthlyIncome': 150000, 'incomeCV': 0.02, 'expenseRatio': 0.15, 
    'emiRatio': 0.05, 'avgMonthlyBalance': 100000, 'bounceCount': 0, 'accountAgeMonths': 60
}

risky_customer = {
    'avgMonthlyIncome': 30000, 'incomeCV': 0.40, 'expenseRatio': 0.80, 
    'emiRatio': 0.60, 'avgMonthlyBalance': 500, 'bounceCount': 3, 'accountAgeMonths': 6
}

print('--- Elite Customer Explanation ---')
print(json.dumps(get_decision_explanation(elite_customer), indent=2))

print('\n--- Risky Customer Explanation ---')
print(json.dumps(get_decision_explanation(risky_customer), indent=2))

--- Elite Customer Explanation ---
{
  "PD": {
    "value": 0.0,
    "top_factors": [
      "Monthly Expense Burden (-5.416)",
      "Existing Debt Commitments (EMI) (-2.009)",
      "Historical Payment Bounces (-0.309)"
    ]
  },
  "Anomaly": {
    "score": 0.0955,
    "top_factors": [
      "Monthly Expense Burden (-1.447)",
      "Existing Debt Commitments (EMI) (-0.774)",
      "Liquidity Reserve (+0.350)"
    ]
  },
  "RiskLabel": {
    "label": "LOW",
    "drivers": [
      "Probability of Default Signal (+0.359)",
      "Monthly Expense Burden (+0.113)",
      "Existing Debt Commitments (EMI) (+0.093)"
    ]
  },
  "HybridScore": {
    "value": 585.6,
    "factors": [
      "Monthly Expense Burden (+30.536)",
      "Existing Debt Commitments (EMI) (+27.899)",
      "Historical Payment Bounces (+22.981)"
    ]
  },
  "RL_Recommendation": {
    "action": "REJECT",
    "rationales": [
      "Anomaly Signal Intensity (-28.286)",
      "Probability of Default Signal (+16.373)",
    

## 🟢 Elite Customer – Detailed Model Explanation

---
### ✅ 1️⃣ Probability of Default (PD)

```json
"value": 0.0
```

### 📌 Interpretation

The system estimates **almost zero probability of default**.

#### 🔍 Key Contributing Factors

| Feature | SHAP Impact | Interpretation |
|----------|-------------|---------------|
| Monthly Expense Burden | -5.444 | Low expense ratio reduces risk |
| Existing Debt Commitments (EMI) | -1.931 | Low EMI burden reduces risk |
| Historical Payment Bounces | -0.331 | Clean repayment history |

#### 🧠 Meaning

- Customer spends a small portion of income  
- Customer has low debt obligations  
- Customer has no payment bounce history  

👉 Strong financial discipline  
👉 Extremely low credit risk  

---

### ✅ 2️⃣ Anomaly Detection (Behavioral Check)

```json
"score": 0.0955
```

### 📌 Interpretation

- Positive score → Behavior classified as **normal**
- No unusual financial pattern detected

#### 🔍 Key Contributing Factors

| Feature | SHAP Impact | Interpretation |
|----------|-------------|---------------|
| Monthly Expense Burden | -1.447 | Stable spending behavior |
| Existing Debt Commitments (EMI) | -0.774 | Normal repayment pattern |
| Liquidity Reserve | +0.350 | Strong cash balance |

#### 🧠 Meaning

- Expense and EMI behavior look stable  
- Adequate liquidity buffer  
- No suspicious pattern  

👉 Customer behavior appears normal  

---

### ✅ 3️⃣ Risk Classification

```json
"label": "LOW"
```

### 📌 Interpretation

Customer is classified as **LOW risk**.

#### 🔍 Supporting Signals

| Signal | SHAP Contribution |
|---------|------------------|
| Probability of Default Signal | +0.359 |
| Monthly Expense Burden | +0.113 |
| Existing Debt Commitments (EMI) | +0.093 |

#### 🧠 Meaning

Combined model signals strongly support LOW risk classification.

👉 Credit profile is stable and safe  

---

### ✅ 4️⃣ Hybrid Credit Score

```json
"value": 606.3
```

### 📌 Interpretation

Score 606.3 indicates a **good credit profile**.

#### 🔍 Major Score Drivers

| Feature | Score Contribution |
|----------|------------------|
| Monthly Expense Burden | +30.253 |
| Existing Debt Commitments (EMI) | +22.068 |
| Historical Payment Bounces | +19.715 |

#### 🧠 Meaning

- Low expenses boosted score  
- Low EMI increased creditworthiness  
- Clean payment history added strong positive impact  

👉 Strong borrower profile  

---

## ⚠ 5️⃣ RL Recommendation

```json
"action": "REJECT"
```

#### 🔍 RL Rationale Signals

| Signal | Contribution |
|---------|-------------|
| Anomaly Signal Intensity | -33.915 |
| Probability of Default Signal | +20.430 |
| Consolidated Credit Score | +0.000 |

#### 🧠 Interpretation

- RL policy is overly sensitive to anomaly signal  
- PD and credit score signals are not sufficiently weighted  
- Indicates **policy calibration imbalance**

👉 This is a **policy tuning issue**, not a data quality issue  
